In [20]:
import numpy as np
import rebound
import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
from matplotlib.lines import Line2D
from matplotlib.patches import Rectangle
from matplotlib.legend_handler import HandlerBase

class HandlerColormap(HandlerBase):
    def __init__(self, cmap, num_stripes=8, **kw):
        HandlerBase.__init__(self, **kw)
        self.cmap = cmap
        self.num_stripes = num_stripes
    def create_artists(self, legend, orig_handle, 
                       xdescent, ydescent, width, height, fontsize, trans):
        stripes = []
        for i in range(self.num_stripes):
            s = Rectangle([xdescent + i * width / self.num_stripes, ydescent], 
                          width / self.num_stripes, 
                          height, 
                          fc=self.cmap((2 * i + 1) / (2 * self.num_stripes)), 
                          transform=trans)
            stripes.append(s)
        return stripes
cm = plt.cm.get_cmap('plasma_r')
v_lim = [5, 8]

In [21]:
hip = np.genfromtxt('hipparcos-bright-result.csv', delimiter=',',skip_header=1,usecols=[1])
visor,magC,adu,npix,bkgnd,streak,rate = np.genfromtxt("mags-streaks.dat",usecols=(1,2,3,4,5,6,7),unpack=True)
gmag = -2.5*np.log10(adu - npix*bkgnd) + magC -2.5 * np.log10(30* rate/streak)

In [22]:
MEarth = 5.97e24
REarth = 6378.135e3

constellations = {
    "Starlink": [ {'NPLANES':7178,'SATPP':1,'INC':30,'ALT':328},
        {'NPLANES':7178,'SATPP':1,'INC':40,'ALT':334},
        {'NPLANES':7178,'SATPP':1,'INC':53,'ALT':345},
        {'NPLANES':40,'SATPP':50,'INC':96.9,'ALT':360},
        {'NPLANES':1998,'SATPP':1,'INC':75,'ALT':373},
        {'NPLANES':4000,'SATPP':1,'INC':53,'ALT':499},
        {'NPLANES':12,'SATPP':12,'INC':148,'ALT':604},
        {'NPLANES':18,'SATPP':18,'INC':115.7,'ALT':614},
        {'NPLANES':2547,'SATPP':1,'INC':53,'ALT':345.6},
        {'NPLANES':2478,'SATPP':1,'INC':48,'ALT':340.8},
        {'NPLANES':2493,'SATPP':1,'INC':42,'ALT':335.9},
        {'NPLANES':32,'SATPP':50,'INC':53,'ALT':550},
        {'NPLANES':72,'SATPP':22,'INC':53.2,'ALT':540},
        {'NPLANES':36,'SATPP':20,'INC':70,'ALT':570},
        {'NPLANES':6,'SATPP':58,'INC':97.6,'ALT':560},
        {'NPLANES':4,'SATPP':43,'INC':97.6,'ALT':560.1},],
    "OneWeb": [ {'NPLANES':18,'SATPP':40,'INC':87.9,'ALT':1200},
        {'NPLANES':36,'SATPP':49,'INC':87.9,'ALT':1200},
        {'NPLANES':32,'SATPP':72,'INC':40,'ALT':1200},
        {'NPLANES':32,'SATPP':72,'INC':55,'ALT':1200},],
    "StarNet/GW": [ {'NPLANES':16,'SATPP':30,'INC':85,'ALT':590},
        {'NPLANES':40,'SATPP':50,'INC':50,'ALT':600},
        {'NPLANES':60,'SATPP':60,'INC':55,'ALT':508},
        {'NPLANES':48,'SATPP':36,'INC':30,'ALT':1145},
        {'NPLANES':48,'SATPP':36,'INC':40,'ALT':1145},
        {'NPLANES':48,'SATPP':36,'INC':50,'ALT':1145},
        {'NPLANES':48,'SATPP':36,'INC':60,'ALT':1145},],
    "Kuiper": [ {'NPLANES':34,'SATPP':34,'INC':51.9,'ALT':630},
        {'NPLANES':36,'SATPP':36,'INC':42,'ALT':610},
        {'NPLANES':28,'SATPP':28,'INC':33,'ALT':509},],
    }


def add_to_simulation(sim, ICs, debug=False):
    for IC in ICs:
        nplanes=IC['NPLANES']
        nsat=IC['SATPP']
        a = IC['ALT']*1000.+REarth

        Omegas = np.linspace(0.,2.*np.pi,nplanes)
        for i, Omega in enumerate(Omegas):
            # 5 percent jitter
            Ms = np.linspace(0.,2.*np.pi,nsat)+ 2.*np.pi/nsat*0.25*np.random.normal(size=nsat)
            for j, M in enumerate(Ms):
                sim.add(M=M, a=a, omega=0, e=0, Omega=Omega, inc=IC['INC']*np.pi/180.)
                if debug and sim.N>100:
                    return
def rotY(xyz,alpha):
    c, s = np.cos(alpha), np.sin(alpha)
    M = np.array([[c,0,-s],[0,1,0],[s,0,c]])
    return xyz @ M
def rotZ(xyz,alpha):
    c, s = np.cos(alpha), np.sin(alpha)
    M = np.array([[c,-s,0],[s,c,0],[0,0,1]])
    return xyz @ M
def length_of_night(timeOfYear,latitude, p=0):
    # https://www.ikhebeenvraag.be/mediastorage/FSDocument/171/Forsythe+-+A+model+comparison+for+daylength+as+a+function+of+latitude+and+day+of+year+-+1995.pdf
    # p=18 for astronomical twilight
    theta = 2.*np.arctan(0.9671396*np.tan(-timeOfYear/2.+np.pi/4.))
    phi = np.arcsin(0.39795*np.cos(theta))
    return 24./np.pi * np.arccos((np.sin(p*np.pi/180.)+np.sin(latitude*np.pi/180.)*np.sin(phi))/(np.cos(latitude*np.pi/180.)*np.cos(phi)))
   
AIRMASSCOR = 1
airmassCoeff=0.2 
def getAirmass(z):
    X = 1./(np.cos(z) + 0.50572*(6.07995+90-z*180/np.pi)**(-1.6364))  # Kasten and Young (1989)
    #X = 1./np.cos(z) * (1-0.0012*np.tan(z)**2)  # Young and Irvine (1967)
    return X

def get_stereographic_data(sims, latitude, month, hour):
    albedo=0.3
    area=4
    # latitude in degrees
    # month in months from spring euquinox
    # hours in hours since midnight
    latitude = latitude/180.*np.pi 
    tilt = 23.4*np.sin(month/6.*np.pi)/180.*np.pi
    hour = hour/12.*np.pi
    xy, mag = [], []     
    for name in sims:
        sim = sims[name]
        sun = np.array([-1.4959787e+11,0,0]) # in m
        sun = rotY(sun, tilt)
        sun_n = sun/np.linalg.norm(sun)

        obs = np.array([REarth, 0, 0])
        obs = rotY(obs, -latitude)
        obs = rotZ(obs, hour)
        obs_n = obs/np.linalg.norm(obs)

        xyz = np.zeros((sim.N,3),dtype="float64")
        sim.serialize_particle_data(xyz=xyz)
        xyz = xyz[1:] # remove earth


        lit = np.linalg.norm(np.cross(xyz,sun_n),axis=1)>REarth

        xyz = xyz[lit]

        xyz_n = xyz/np.linalg.norm(xyz,axis=1)[:,np.newaxis]
        xyz_r = xyz - obs
        xyz_rd = np.linalg.norm(xyz_r,axis=1)
        xyz_rn = xyz_r/xyz_rd[:,np.newaxis]

        phase = np.arccos(np.clip(np.dot(xyz_rn, -sun_n), -1.0, 1.0)) # assume sun is in -x direction
        #angle = np.abs(np.arccos(dotprod/(obslen*satlen)))
        elevation = (np.pi/2.-np.arccos(np.dot(xyz_rn,obs_n)))/np.pi*180.

        fac1 = 2/(3*np.pi**2)
        magV = -26.74 -2.5*np.log10(fac1 * area * albedo * ( (np.pi-phase)*np.cos(phase) + np.sin(phase) ) ) + 5 * np.log10(xyz_rd)


        xyz = rotZ(xyz, -hour)
        xyz = rotY(xyz, latitude)
        xyz_r = xyz - np.array([REarth, 0, 0])
        xyz_rd = np.linalg.norm(xyz_r,axis=1)
        xyz_rn = xyz_r/xyz_rd[:,np.newaxis]
        
        airmass=getAirmass((elevation)*np.pi/180.)
        #print(elevation,airmass)
        magV+=airmassCoeff*airmass*AIRMASSCOR
        magV+=np.random.normal(0.,0.2,size=len(magV))

        elevation_cut = 0
        xyz_rn = xyz_rn[elevation>elevation_cut]
        magV = magV[elevation>elevation_cut]

        xy.append(xyz_rn[:,1:3]/(1.+xyz_rn[:,0,np.newaxis]))
        mag.append(magV)
    if len(xy)>0:
        return np.concatenate(xy), np.concatenate(mag) 
    else:
        return None, None


def get_simulations(add_constellations=None, use_cache=True):
    if add_constellations is None:
        add_constellations = constellations.keys() # all constellations
    sims = {}
    for c in add_constellations:
        if c not in constellations:
            raise RuntimeError("Constellation %s not found."%c)
        sim = None
        if use_cache:
            filename = "mega_"+"".join(x for x in c if x.isalnum())+".bin"
            try:
                sim = rebound.Simulation(filename)
            except:
                # need to create simulation
                pass
        if sim is None:
            sim = rebound.Simulation()
            sim.G = 6.67430e-11
            sim.add(m=MEarth)
            sim.N_active = 1
            add_to_simulation(sim, constellations[c])
            if use_cache:
                sim.save(filename)
        sims[c] = sim
    return sims

sims = get_simulations()

In [23]:
print("stars <4.",len(hip[hip<4.]))
print("stars <5.",len(hip[hip<5.]))
print("stars <6.5.",len(hip[hip<6.5]))
print("stars <7.",len(hip[hip<7.]))
for latitude in [60,50,40,30,20]:
    for month in [3,0]:
        for hour in [0,-3]:
            for maglim in [5.,6.5,7.]:
                xy, mag = get_stereographic_data(sims, latitude=latitude, month=month, hour=hour)
                print("lat ",latitude,", time ",hour,", month ",month,", <",maglim,len(mag[mag<maglim]))



stars <4. 516
stars <5. 1608
stars <6.5. 8789
stars <7. 15404
lat  60 , time  0 , month  3 , < 5.0 40
lat  60 , time  0 , month  3 , < 6.5 420
lat  60 , time  0 , month  3 , < 7.0 651
lat  60 , time  -3 , month  3 , < 5.0 20
lat  60 , time  -3 , month  3 , < 6.5 461
lat  60 , time  -3 , month  3 , < 7.0 758
lat  60 , time  0 , month  0 , < 5.0 0
lat  60 , time  0 , month  0 , < 6.5 10
lat  60 , time  0 , month  0 , < 7.0 28
lat  60 , time  -3 , month  0 , < 5.0 1
lat  60 , time  -3 , month  0 , < 6.5 79
lat  60 , time  -3 , month  0 , < 7.0 155
lat  50 , time  0 , month  3 , < 5.0 51
lat  50 , time  0 , month  3 , < 6.5 330
lat  50 , time  0 , month  3 , < 7.0 522
lat  50 , time  -3 , month  3 , < 5.0 107
lat  50 , time  -3 , month  3 , < 6.5 626
lat  50 , time  -3 , month  3 , < 7.0 935
lat  50 , time  0 , month  0 , < 5.0 0
lat  50 , time  0 , month  0 , < 6.5 0
lat  50 , time  0 , month  0 , < 7.0 1
lat  50 , time  -3 , month  0 , < 5.0 0
lat  50 , time  -3 , month  0 , < 6.5 35
lat

In [24]:
#all sky flux/sq. degree
sum(100**((-hip)/5.)*3631.)/20626.5

26.55461426681228

In [25]:
#all sky flux/sq. degree for satellites
xy, mag = get_stereographic_data(sims, latitude=50, month=3, hour=-3)
sum(100**((-mag)/5.)*3631.)/20626.5

1.0026694759293073